In [1]:
import dash
from dash import html
from dash.dependencies import Input, Output
from markupsafe import Markup
from dash import dcc
import plotly.express as px
import pandas as pd
import numpy as np
from faker import Faker
import plotly.graph_objs as go
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium import Map, Circle
from folium.plugins import HeatMapWithTime, HeatMap, Geocoder, Draw, MeasureControl
from folium.utilities import JsCode
from folium.features import GeoJsonPopup
from folium.plugins import TimeSliderChoropleth
from folium import plugins
import pickle
import json
from jinja2 import Template

In [2]:
with open('dic_all_predictions500.pkl', 'rb') as f:
    loaded_all_predictions = pickle.load(f)
with open('all_aeds.pkl', 'rb') as f:
    loaded_aeds = pickle.load(f)

In [3]:
class DataPreparationIntervention:
    def __init__(self, dictionary):
        self.dictionary=dictionary
        self.quantify_AED()
        self.create_heatmap_data()
    def transform_dataframe(self,df):
        status_mapping = {'Does not require AED': 0.1, 'Maybe requires AED': 0.3, 'Requires AED': 1}
        for hour in range(24):
            column_name_old = f'hour_{hour}'
            column_name_new = f'value_hour{hour}'
            df[column_name_new] = df[column_name_old].map(status_mapping)
            df = df.drop(columns=column_name_old)
        return df
    def quantify_AED(self):
        for city, days in self.dictionary.items():
            for day, df in days.items():
                self.dictionary[city][day] = self.transform_dataframe(df)
    def extract_heatmap_data(self,df):
        self.heat_data = []
        for hour in range(24):
            column_name = f'value_hour{hour}'
            hourly_data = df[['lat', 'lon', column_name]].values.tolist()
            self.heat_data.append(hourly_data)
        return self.heat_data
    def create_heatmap_data(self):
        self.heatmap_data = {}
        for city, days in self.dictionary.items():
            dic_per_city ={}
            for day, df in days.items():
                dic_per_city[day] =self.extract_heatmap_data(df)
            self.heatmap_data[city] = dic_per_city

In [4]:
class DataPreparationAED:
    def __init__(self, dictionary):
        self.dictionary=dictionary
        self.quantify_location()
    def quantify_location(self):
        status_mapping={'inside less accessible': 120,'inside access': 150,'outside public access': 200}
        for city, days in self.dictionary.items():
            for day, df in days.items():
                self.dictionary[city][day]['buffer_distance']= self.dictionary[city][day]['location_cleaned'].map(status_mapping)
                self.dictionary[city][day]= self.dictionary[city][day].drop(columns=['location_cleaned'])


In [5]:
prepareInt = DataPreparationIntervention(loaded_all_predictions)
prepareAED = DataPreparationAED(loaded_aeds)

In [6]:
heatmap_data = prepareInt.heatmap_data
AED_data = prepareAED.dictionary

In [7]:
print(heatmap_data.keys())
print(AED_data.keys())

dict_keys(['Brussels', 'Antwerpen', 'Brugge', 'Gent', 'Hasselt', 'Leuven', 'Mons', 'Liège', 'Charleroi', 'Namur', 'Arlon'])
dict_keys(['Antwerp', 'Bruges', 'Brussels', 'Ghent', 'Hasselt', 'Leuven', 'Liege', 'Mons', 'Namur', 'Charleroi', 'Arlon'])


In [8]:
AED_data['Antwerp']

{'Mon':            lat       lon  opening_hour  closing_hour  buffer_distance
 0    51.227361  4.404384           9.0          16.0              120
 1    51.228761  4.438235           9.0          16.0              120
 2    51.246664  4.402453           9.0          16.0              120
 3    51.208432  4.421169           9.0          16.0              120
 4    51.214577  4.413205           9.0          16.0              120
 ..         ...       ...           ...           ...              ...
 438  51.189434  4.451715           9.0          16.0              120
 439  51.212274  4.406393           9.0          16.0              150
 440  51.315844  4.332686           8.0          16.0              120
 441  51.041171  4.469566           9.0          17.0              150
 442  51.246664  4.402453           8.0          16.0              120
 
 [443 rows x 5 columns],
 'Tue':            lat       lon  opening_hour  closing_hour  buffer_distance
 0    51.227361  4.404384           

In [9]:
cities = {'Antwerpen': 'Antwerp', 
                  'Brugge': 'Bruges',
                  'Brussels': 'Brussels', 
                  'Gent': 'Gent',
                  'Hasselt': 'Hasselt',
                  'Leuven': 'Leuven',
                  'Liège':'Liege',
                  'Mons': 'Mons',
                  'Namur': 'Namur',
                  'Charleroi': 'Charleroi',
                  'Arlon': 'Arlon'}

In [10]:
aed_df_city = AED_data[cities['Antwerpen']] 

In [18]:
#this works!!!!

app = dash.Dash(__name__)

def makemap(city, dayaed, dayheat):
    try:
        map = folium.Map(location=[50.5, 4.3517], zoom_start=8)
        cities = {'Antwerpen': 'Antwerp', 
                  'Brugge': 'Bruges',
                  'Brussels': 'Brussels', 
                  'Gent': 'Ghent',
                  'Hasselt': 'Hasselt',
                  'Leuven': 'Leuven',
                  'Liège':'Liege',
                  'Mons': 'Mons',
                  'Namur': 'Namur',
                  'Charleroi': 'Charleroi',
                  'Arlon': 'Arlon'}
        
        aed_df_city = AED_data[cities[city]] 
        aed_df = aed_df_city[dayaed]
        for index, row in aed_df.iterrows():
            circle = folium.Circle(
                location=[row['lat'], row['lon']],
                radius=row['buffer_distance'], 
                color='black', 
                fill=True,
                fill_color='black', 
                fill_opacity=0.7,
                popup=folium.Popup(f"open from: {row['opening_hour']}h to {row['closing_hour']}h")
            ).add_to(map)
            
        HeatMapWithTime(
            data=heatmap_data[city][dayheat],
            position= 'topright',
            gradient={0: 'green', 0.2: 'yellow', 1:'red'},
            radius=20
        ).add_to(map)

        Geocoder().add_to(map)

        Draw(export=True).add_to(map)

        MeasureControl().add_to(map)
        
        
        return map._repr_html_()
    except Exception as e:
        return f"<p>Error creating map: {str(e)}</p>"

app.layout = html.Div([
    dcc.RadioItems(
        id='city-choice',
        options=[{'label': city, 'value': city} for city in ['Antwerpen', 'Brugge', 'Brussels',  
                                                             'Gent', 'Hasselt', 'Leuven', 'Liège', 
                                                             'Mons', 'Namur', 'Charleroi', 'Arlon']],
        value='Brussels',
        inline=True
    ),
    dcc.RadioItems(
        id='day-choice',
        options=[{'label': day, 'value': day} for day in ['Monday', 'Tuesday', 
                                                          'Wednesday', 'Thursday', 
                                                          'Friday', 'Saturday', 'Sunday']],
        value='Friday',
        inline=True
    ),
    html.Iframe(
        id='folium-map',
        srcDoc=makemap('Brussels', 'Fri', 'Friday'),
        width='100%',
        height='600px'
    ),
])

@app.callback(
    Output('folium-map', 'srcDoc'),
    [Input('city-choice', 'value'),
     Input('day-choice', 'value')]
)
def update_map(selected_city, selected_day):
    try:
        dayaed = selected_day[:3]
        dayheat = selected_day
        return makemap(selected_city, dayaed, dayheat)
    except Exception as e:
        return f"<p>Error updating map: {str(e)}</p>"

if __name__ == '__main__':
    app.run_server(debug=True)